Check if saving and everything else has been correctly done after run the script for prestim later none filter on eCortex

In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from mne.datasets import sample
from mne.decoding import cross_val_multiscore, LinearModel, GeneralizingEstimator, Scaler, \
                         Vectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, StratifiedShuffleSplit, \
                                    RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import argparse

# Set args

In [4]:
class arguments():
    SAVE_EPOCH_ROOT = '../../data/version5.2/preprocessed/epochs/'
    SAVE_RESULT_ROOT = '../../results/temp_gen/eCortex/laterNonePre/'
    cond_filter ='none' # {none,non_symm}
    cond_block ='later' #{early,later}
    cond_time = 'prestim' #{prestim,poststim}
    cond_decoding = 'none' #{none,removeevoked,resampled}
    subj_num = 1
    applyBaseline_bool = 0
    pre_tmin = -0.4
    pre_tmax = 0.05
    post_tmin = 0.05
    post_tmax = 0.45
    num_classes = 2
    normalization_type = 'normal'# {normal,lstmPaper}
    gen_rand_perm = 0
    null_max_iter = 10000
    loop_null_iter = 5
    gen_decoder_scores = 1
    n_splits = 5
    random_state = 42 
    max_iter = 10000
    n_jobs = 1
    scoring = 'roc_auc'
    
args = arguments()

# load nested list results using pickle

In [8]:
main_ptrn=1

fn_str_sbj='scores_timeGen_%sBlocks_%sFilter_%s_decod%s_bsline%s_Subj_%s' %(args.cond_block, args.cond_filter, args.cond_time, \
                                                               args.cond_decoding, args.applyBaseline_bool, \
                                                               args.subj_num)

fn_str = args.SAVE_RESULT_ROOT + 'avgP%s_' %(main_ptrn) + fn_str_sbj

In [5]:

with open(fn_str, 'rb') as f:
    sc_subj = pickle.load(f)

NameError: name 'fn_str' is not defined